In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [34]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [35]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [36]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [37]:
pd.read_sql(
    '''
    SELECT sum(case when CRIM is null then 1 else 0 end) as CRIM_cnt_null, 
        sum(case when ZN is null then 1 else 0 end) as ZN_cnt_null, 
        sum(case when INDUS is null then 1 else 0 end) as INDUS_cnt_null, 
        sum(case when CHAS is null then 1 else 0 end) as CHAS_cnt_null, 
        sum(case when NOX is null then 1 else 0 end) as NOX_cnt_null
    FROM boston
    ''',
    con,
)

,CRIM_cnt_null,ZN_cnt_null,INDUS_cnt_null,CHAS_cnt_null,NOX_cnt_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [40]:
pd.read_sql(
    '''
    SELECT count(distinct CRIM) as CRIM,
        count(distinct ZN) as ZN, 
        count(distinct INDUS) as INDUS, 
        count(distinct CHAS) as CHAS, 
        count(distinct NOX) as NOX
    FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [145]:
# для расчет медианы посчитала для каждого значения нужного поля сумму этого и следующего жначения деленное на два:
# пример: sum(ZN) over(order by ZN ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0
# это для случая, когда количесмтво значений четное и чтобы высчитать медиану, надо взять среднее двух значений посередине
# в этом случае нам надо будет взять рассчитанное значение из строки <кол-во всех строк>/2
# если кол-во строк нечетное, тогда берем значение поля из строчки round(cnt_all/2,0) - это будет ровно середина
# далее я эту медиану сравниваю с минимальным значением этого поля.

pd.read_sql(
    '''
    SELECT distinct
               
                case when case when rownum_crim=round(cnt_all/2,0) then case when cnt_all%2=0
                    then crim_median else crim end end = min_crim then 'CRIM, ' else '' end
                ||
                case when case when rownum_ZN=round(cnt_all/2,0) then case when cnt_all%2=0
                    then ZN_median else ZN end end = min_ZN then 'ZN, ' else '' end
                ||
                case when case when rownum_INDUS=round(cnt_all/2,0) then case when cnt_all%2=0
                    then INDUS_median else INDUS end end= min_INDUS then 'INDUS, ' else '' end
                ||
                case when case when rownum_CHAS=round(cnt_all/2,0) then case when cnt_all%2=0
                    then CHAS_median else CHAS end end = min_CHAS then 'CHAS, ' else '' end
                ||
                case when case when rownum_NOX=round(cnt_all/2,0) then case when cnt_all%2=0
                    then NOX_median else NOX end end=min_NOX then 'NOX' else '' end
                as column_names
           from(     
    select *,
   sum(crim) over(order by crim ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0 as crim_median,
   min(crim) over() as min_crim,
   sum(ZN) over(order by ZN ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0 as zn_median,
   min(ZN) over() as min_zn,
   sum(INDUS) over(order by INDUS ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0 as INDUS_median,
   min(INDUS) over() as min_INDUS,
   sum(CHAS) over(order by CHAS ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0 as CHAS_median,
   min(CHAS) over() as min_CHAS,
   sum(NOX) over(order by NOX ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING)/2.0 as NOX_median,
   min(NOX) over() as min_NOX,
   row_number() over(order by crim) as rownum_crim,
   row_number() over(order by ZN) as rownum_ZN,
   row_number() over(order by INDUS) as rownum_INDUS,
   row_number() over(order by CHAS) as rownum_CHAS,
   row_number() over(order by NOX) as rownum_NOX,
   count(*) over() as cnt_all

   FROM boston) a
 

    ''',

    con,

)

,column_names
0,
1,"CHAS,"
2,"ZN,"


Вывод: меньше половины объктов не граничат с рекой

4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [64]:

pd.read_sql(
    '''
    select lstat,medv,rn_lstat from (
    SELECT *,
        dense_rank() over (order by lstat) as rn_lstat,
        rank() over(order by medv desc) as rn_medv
    FROM boston
    ) a
    where rn_medv=1
    order by medv desc
    ''',
    con,
)

,LSTAT,MEDV,rn_lstat
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,14
6,3.26,50.0,15
7,3.32,50.0,16
8,3.70,50.0,23
9,3.73,50.0,24


5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [72]:

pd.read_sql(
    '''
    SELECT case when chas=1 then 'граничит с ренкой' else 'не граничит с рекой' end as chas,
    round(avg(medv)*1000,0) as avg_cost
    FROM boston
    group by 1
    ''',
    con,
)
#вывод: возле реки средняя стоимость домов выше

,chas,avg_cost
0,граничит с ренкой,28440.0
1,не граничит с рекой,22094.0


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [147]:
pd.read_sql(
  '''
select case when sum(avg_crim*mn)>0 then 'CRIM, ' else '' end
     ||
     case when sum(avg_zn*mn)>0 then 'ZN, ' else '' end
     ||
     case when sum(avg_indus*mn)>0 then 'INDUS, ' else '' end
      ||
      case when sum(avg_nox*mn)>0 then 'NOX, ' else '' end
      as column_names

   from (
           SELECT case when chas=1 then 1 else -1 end as mn,
        --        avg(crim) as avg_crim, avg(nz) as avg_nz,avg(indus) as avg_indus, avg(nox) as avg_nox
                avg(ZN) as avg_ZN,
                avg(INDUS) as avg_INDUS,
                avg(CRIM) as avg_CRIM,
                avg(NOX) as avg_NOX
            FROM boston
            group by 1
    ) a
    ''',

    con,

)


 

,column_names
0,"INDUS, NOX,"


7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [144]:

pd.read_sql(
    '''
    select part, avg(indus) as INDUS_avg, avg(nox) as NOX_avg from (
    SELECT NTILE(10) over(order by indus,nox) as part,indus,nox FROM boston
    ) a
    group by 1
    ''',
    con,
)

,part,INDUS_avg,NOX_avg
0,1,2.016078,0.438378
1,2,3.645294,0.494129
2,3,5.246275,0.448235
3,4,6.471765,0.470839
4,5,8.256078,0.531510
5,6,10.749216,0.515176
6,7,17.245800,0.564920
7,8,18.100000,0.684880
8,9,18.307200,0.714620
9,10,21.938800,0.692860


In [ ]:
Вывод: чем выше доля промышденной завтройки, тем выше концентрация окстдов ахота (загрязненный воздух)